In [ ]:
import numpy as np
import pandas as pd


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
attr = pd.read_csv('list_attr_celeba.csv')
attr.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [ ]:
attr.replace(-1, 0, inplace = True)

In [ ]:
import os
latent_vectors = sorted(os.listdir('latent vectors'))
l_vectors = []
for l in latent_vectors:
    l_v = int(l[:6])
    l_vectors.append(l_v)
print(len(l_vectors))

drop_rows = []
for i in range(1, 4401):
    if i not in l_vectors:
        drop_rows.append(i-1)
        

3755
646
-3019


In [ ]:
X = attr.iloc[:4401, 1:]

In [ ]:
X = X.drop(drop_rows, axis = 0)

In [ ]:
X = X.values

In [ ]:
X.shape

(3755, 40)

In [ ]:
# loading target vaiables
def load_target_variables(path_dir: str):
    Y = []
    for latents in sorted(os.listdir(path_dir)):
        y = np.load(path_dir + '/' +latents).reshape(-1)
        Y.append(y)
    
    return Y
    

In [ ]:
Y = load_target_variables('latent vectors')

In [ ]:
Y = np.array(Y)

In [ ]:
Y.shape

(3755, 9216)

In [ ]:
# normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.9)

In [ ]:
X_train.shape

(3379, 40)

In [ ]:
Y_train.shape

(3379, 9216)

In [ ]:
import keras

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
def create_model():
    latent_optimization_model = Sequential()
    latent_optimization_model.add(Dense(128, input_dim = 40, activation= 'relu'))
    latent_optimization_model.add(Dense(256, activation = 'relu'))
    latent_optimization_model.add(Dense(256, activation = 'relu'))
    latent_optimization_model.add(Dropout(0.2))
    latent_optimization_model.add(Dense(1024, activation = 'relu'))
    latent_optimization_model.add(Dense(1024, activation = 'relu'))
    latent_optimization_model.add(Dense(2048, activation = 'relu'))
    latent_optimization_model.add(Dropout(0.2))
    latent_optimization_model.add(Dense(2048, activation = 'relu'))
    latent_optimization_model.add(Dense(4096, activation = 'relu'))
    latent_optimization_model.add(Dense(4096, activation = 'relu'))
    latent_optimization_model.add(Dropout(0.2))
    latent_optimization_model.add(Dense(9216, activation = 'relu'))
    latent_optimization_model.add(Dense(9216, activation = 'relu'))
    latent_optimization_model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return latent_optimization_model

In [ ]:
#latent_optimization_model = Sequential()

In [ ]:
# latent_optimization_model.add(Dense(128, input_dim = 40, activation= 'relu'))
# latent_optimization_model.add(Dense(256, activation = 'relu'))
# latent_optimization_model.add(Dense(256, activation = 'relu'))
# latent_optimization_model.add(Dropout(0.2))
# latent_optimization_model.add(Dense(1024, activation = 'relu'))
# latent_optimization_model.add(Dense(1024, activation = 'relu'))
# latent_optimization_model.add(Dense(2048, activation = 'relu'))
# latent_optimization_model.add(Dropout(0.2))
# latent_optimization_model.add(Dense(2048, activation = 'relu'))
# latent_optimization_model.add(Dense(4096, activation = 'relu'))
# latent_optimization_model.add(Dense(4096, activation = 'relu'))
# latent_optimization_model.add(Dropout(0.2))
# latent_optimization_model.add(Dense(9216, activation = 'relu'))
# latent_optimization_model.add(Dense(9216, activation = 'relu'))

In [ ]:
# latent_optimization_model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
#i1_model = latent_optimization_model.fit(X_train, Y_train, epochs = 150, batch_size=44)

In [ ]:
#latent_optimization_model.save('model_400iter_450sample')

In [ ]:
# for reloading the model
# keras.models.load_model('model_150iter_450sample')

In [ ]:
model_3750 = create_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs= {}):
        if (logs.get('accuracy') > 0.8):
            print("\n Reached 80% accuracy hence cancelling the training!")
            self.model.stop_training=True
            s = "model_" + str(epoch) + "iter_3750_sample"
            self.model.save(s)

In [ ]:
callbacks = myCallback()

In [ ]:
model_3750.fit(X_train, Y_train, epochs = 250, batch_size=44, callbacks= [callbacks])


Epoch 1/250
3379/3379 [==============================] - 193s 57ms/step - loss: 0.0193 - accuracy: 0.2622
Epoch 2/250
3379/3379 [==============================] - 159s 47ms/step - loss: 0.0182 - accuracy: 0.3164
Epoch 3/250
3379/3379 [==============================] - 152s 45ms/step - loss: 0.0182 - accuracy: 0.3164
Epoch 4/250
3379/3379 [==============================] - 146s 43ms/step - loss: 0.0182 - accuracy: 0.3164
Epoch 5/250
3379/3379 [==============================] - 145s 43ms/step - loss: 0.0182 - accuracy: 0.3164
Epoch 6/250
3379/3379 [==============================] - 145s 43ms/step - loss: 0.0181 - accuracy: 0.3164
Epoch 7/250
3379/3379 [==============================] - 145s 43ms/step - loss: 0.0180 - accuracy: 0.3164
Epoch 8/250
3379/3379 [==============================] - 29855s 9s/step - loss: 0.0180 - accuracy: 0.3164
Epoch 9/250
3379/3379 [==============================] - 149s 44ms/step - loss: 0.0179 - accuracy: 0.3164
Epoch 10/250
3379/3379 [=====================

In [ ]:
model_3750.save('model_3750_250Iter')

In [ ]:
X_test.shape

(376, 40)

In [ ]:
y_pred = model_3750.predict(X_test)
y_p = y_pred[4].reshape(-1)

In [ ]:
y_p

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [ ]:
latent_pred = y_p.reshape(1, 18, 512)

In [ ]:
latent_pred.shape

(1, 18, 512)

In [ ]:
for i, latents in enumerate(y_pred):
    latents.reshape(-1).reshape(1,18,512)
    out_path = 'output_vectors/' + str(i)
    np.save(out_path, latents)

In [ ]:
sorted(os.listdir('output_vectors'))

['0.npy',
 '1.npy',
 '10.npy',
 '100.npy',
 '101.npy',
 '102.npy',
 '103.npy',
 '104.npy',
 '105.npy',
 '106.npy',
 '107.npy',
 '108.npy',
 '109.npy',
 '11.npy',
 '110.npy',
 '111.npy',
 '112.npy',
 '113.npy',
 '114.npy',
 '115.npy',
 '116.npy',
 '117.npy',
 '118.npy',
 '119.npy',
 '12.npy',
 '120.npy',
 '121.npy',
 '122.npy',
 '123.npy',
 '124.npy',
 '125.npy',
 '126.npy',
 '127.npy',
 '128.npy',
 '129.npy',
 '13.npy',
 '130.npy',
 '131.npy',
 '132.npy',
 '133.npy',
 '134.npy',
 '135.npy',
 '136.npy',
 '137.npy',
 '138.npy',
 '139.npy',
 '14.npy',
 '140.npy',
 '141.npy',
 '142.npy',
 '143.npy',
 '144.npy',
 '145.npy',
 '146.npy',
 '147.npy',
 '148.npy',
 '149.npy',
 '15.npy',
 '150.npy',
 '151.npy',
 '152.npy',
 '153.npy',
 '154.npy',
 '155.npy',
 '156.npy',
 '157.npy',
 '158.npy',
 '159.npy',
 '16.npy',
 '160.npy',
 '161.npy',
 '162.npy',
 '163.npy',
 '164.npy',
 '165.npy',
 '166.npy',
 '167.npy',
 '168.npy',
 '169.npy',
 '17.npy',
 '170.npy',
 '171.npy',
 '172.npy',
 '173.npy',
 '17

In [ ]:
attr.drop(drop_rows, axis = 0).iloc[644]

image_id               000728.jpg
5_o_Clock_Shadow                0
Arched_Eyebrows                 0
Attractive                      0
Bags_Under_Eyes                 0
Bald                            0
Bangs                           0
Big_Lips                        1
Big_Nose                        1
Black_Hair                      1
Blond_Hair                      0
Blurry                          0
Brown_Hair                      0
Bushy_Eyebrows                  0
Chubby                          0
Double_Chin                     0
Eyeglasses                      0
Goatee                          0
Gray_Hair                       0
Heavy_Makeup                    0
High_Cheekbones                 0
Male                            1
Mouth_Slightly_Open             1
Mustache                        0
Narrow_Eyes                     0
No_Beard                        1
Oval_Face                       0
Pale_Skin                       0
Pointy_Nose                     0
Receding_Hairl

In [ ]:
X[644].sum()

-8.987392608917148